In [6]:
import pandas as pd

# 加载夏季奥运奖牌数据文件
file_path = r'data/summerOly_medal_counts.csv'
data = pd.read_csv(file_path)

# 筛选出至少获得一枚奖牌的记录
data_with_medals = data[(data['Gold'] > 0) | (data['Silver'] > 0) | (data['Bronze'] > 0)]

# 确定每个国家第一次获得奖牌的年份
first_medals = (
    data_with_medals.loc[data_with_medals.groupby('NOC')['Year'].idxmin()]
)

# 筛选出1970年以后首次获奖的国家
first_medals_after_1970 = first_medals[first_medals['Year'] > 1970]

# 只保留关键列：国家代号（NOC）和年份（Year）
first_medals_after_1970 = first_medals_after_1970[['NOC', 'Year']]

# 加载GDP数据文件
gdp_file_path = r'data0/UNdata_Export_20250124_074035299.csv'
gdp_data = pd.read_csv(gdp_file_path)

# 合并两个数据集，按国家（NOC）和年份（Year）匹配
merged_data = pd.merge(first_medals_after_1970, gdp_data, how='left', 
                       left_on=['NOC', 'Year'], right_on=['Country or Area', 'Year'])

# 选择相关的列：国家代号（NOC）、年份（Year）和GDP值（Value）
gdp_for_first_medals = merged_data[['NOC', 'Year', 'Value']]

# 输出结果
print(gdp_for_first_medals)


               NOC  Year        Value
0      Afghanistan  2008   389.604153
1          Albania  2024          NaN
2          Algeria  1984  2469.340135
3          Armenia  1996   516.244396
4       Azerbaijan  1996   399.611791
..             ...   ...          ...
84      Uzbekistan  1996   713.946128
85         Vietnam  2000          NaN
86  Virgin Islands  1988          NaN
87          Zambia  1984   492.296542
88        Zimbabwe  1980  1171.272025

[89 rows x 3 columns]


## 最终版

In [26]:
import pandas as pd

# 加载夏季奥运奖牌数据文件
file_path = r'data/summerOly_medal_counts.csv'
data = pd.read_csv(file_path)

# 筛选出至少获得一枚奖牌的记录
data_with_medals = data[(data['Gold'] > 0) | (data['Silver'] > 0) | (data['Bronze'] > 0)]

# 确定每个国家第一次获得奖牌的年份
first_medals = (data_with_medals.loc[data_with_medals.groupby('NOC')['Year'].idxmin()])


# 筛选出1970年以后首次获奖的国家
first_medals_after_1970 = first_medals[first_medals['Year'] > 1970]

# 只保留关键列：国家代号（NOC）和年份（Year）
first_medals_after_1970 = first_medals_after_1970[['NOC', 'Year']]

"""
first_medals_after_1970_1 = first_medals_after_1970
first_medals_after_1970_1['Year'] = first_medals_after_1970['Year'] - 4 

first_medals_after_1970_2 = first_medals_after_1970
first_medals_after_1970_2['Year'] = first_medals_after_1970['Year'] + 4
print(first_medals_after_1970_2)
"""


# 加载GDP数据文件
gdp_file_path = r'data0/gdp_percentage.csv'
gdp_data = pd.read_csv(gdp_file_path)

# 重命名GDP列
gdp_data = gdp_data.rename(columns={'Value': 'GDP'})

# 合并两个数据集，按国家（NOC）和年份（Year）匹配
merged_data = pd.merge(first_medals_after_1970, gdp_data, how='left', 
                       left_on=['NOC', 'Year'], right_on=['Country or Area', 'Year'])

# 选择相关的列：国家代号（NOC）、年份（Year）和GDP值（GDP）
gdp_for_first_medals = merged_data[['NOC', 'Year', 'GDP_Percentage']].copy()  # 使用 .copy() 来确保这是一个独立的副本

# 加载人口数据文件
population_file_path = r'data0/population_percentage.csv'
population_data = pd.read_csv(population_file_path)

# 将宽表转换为长表，方便按国家和年份匹配
population_data_long = population_data.melt(
    id_vars=['Country'],  # 保留的固定列
    var_name='Year',      # 新的年份列
    value_name='Population'  # 人口数据
)

# 确保 'Year' 列是整数类型
population_data_long['Year'] = population_data_long['Year'].astype(int)

# 遍历 gdp_for_first_medals 的每一行，添加对应的人口数据
for index, row in gdp_for_first_medals.iterrows():
    country_code = row['NOC']  # 国家代号
    year = row['Year']         # 年份
    
    # 找到匹配的国家和年份的人口数据
    population_row = population_data_long[
        (population_data_long['Country'] == country_code) &
        (population_data_long['Year'] == year)
    ]
    
    if not population_row.empty:
        population = population_row['Population'].values[0]
        # 使用 .loc 修改原数据框
        gdp_for_first_medals.loc[index, 'Population'] = population

first_medals_after_1970 = gdp_for_first_medals.rename(columns={'NOC': 'Team','Population':'Population_Percentage'})



# 6. 加载运动员数据
athletes_file_path = r'data/summerOly_athletes.csv'
athletes_data = pd.read_csv(athletes_file_path)

# 7. 查看运动员数据的列名，以确保正确的列名
print("运动员数据列名：", athletes_data.columns)

# 如果列名是 'Name'、'Sport'、'Event'，按需修改
athletes_stats = athletes_data.groupby(['Team', 'Year']).agg(
    athletes_count=('Name', 'nunique'),  # 假设运动员的名字在 'Name' 列
    sports_count=('Sport', 'nunique'),
    events_count=('Event', 'nunique')
).reset_index()

# 8. 将运动员统计数据与第一次获奖年份的数据合并
merged_data = pd.merge(first_medals_after_1970, athletes_stats, on=['Team', 'Year'], how='left')

# 9. 显示结果
print(merged_data)

# 10. 保存最终结果为 CSV 文件
output_file_path = r'final_splitted_data.csv'
merged_data.to_csv(output_file_path, index=False)

# 11. 输出结果确认
print(f"数据已保存到文件：{output_file_path}")



运动员数据列名： Index(['Name', 'Sex', 'Team', 'NOC', 'Year', 'City', 'Sport', 'Event',
       'Medal'],
      dtype='object')
              Team  Year  GDP_Percentage  Population_Percentage  \
0      Afghanistan  2008        0.010874               0.036512   
1          Albania  2024             NaN                    NaN   
2          Algeria  1984        0.325101               0.043100   
3          Armenia  1996        0.029732               0.005366   
4       Azerbaijan  1996        0.023015               0.012706   
..             ...   ...             ...                    ...   
84      Uzbekistan  1996        0.041119               0.038220   
85         Vietnam  2000             NaN                    NaN   
86  Virgin Islands  1988             NaN                    NaN   
87          Zambia  1984        0.064813               0.013333   
88        Zimbabwe  1980        0.139839               0.015397   

    athletes_count  sports_count  events_count  
0              4.0         

## 前4年和后4年

In [21]:
import pandas as pd

# 加载夏季奥运奖牌数据文件
file_path = r'data/summerOly_medal_counts.csv'
data = pd.read_csv(file_path)

# 筛选出至少获得一枚奖牌的记录
data_with_medals = data[(data['Gold'] > 0) | (data['Silver'] > 0) | (data['Bronze'] > 0)]

# 确定每个国家第一次获得奖牌的年份
first_medals = (data_with_medals.loc[data_with_medals.groupby('NOC')['Year'].idxmin()])


# 筛选出1970年以后首次获奖的国家
first_medals_after_1970 = first_medals[first_medals['Year'] > 1970]

# 只保留关键列：国家代号（NOC）和年份（Year）
first_medals_after_1970 = first_medals_after_1970[['NOC', 'Year']]


first_medals_after_1970_1 = first_medals_after_1970
first_medals_after_1970_1['Year'] = first_medals_after_1970['Year'] - 4 

first_medals_after_1970_2 = first_medals_after_1970
first_medals_after_1970_2['Year'] = first_medals_after_1970['Year'] + 4




# 加载GDP数据文件
gdp_file_path = r'data0/gdp_percentage.csv'
gdp_data = pd.read_csv(gdp_file_path)

# 重命名GDP列
gdp_data = gdp_data.rename(columns={'Value': 'GDP'})

# 合并两个数据集，按国家（NOC）和年份（Year）匹配
merged_data = pd.merge(first_medals_after_1970, gdp_data, how='left', 
                       left_on=['NOC', 'Year'], right_on=['Country or Area', 'Year'])

# 合并两个数据集，按国家（NOC）和年份（Year）匹配
merged_data_1 = pd.merge(first_medals_after_1970_1, gdp_data, how='left', 
                       left_on=['NOC', 'Year'], right_on=['Country or Area', 'Year'])

# 合并两个数据集，按国家（NOC）和年份（Year）匹配
merged_data_2 = pd.merge(first_medals_after_1970_2, gdp_data, how='left', 
                       left_on=['NOC', 'Year'], right_on=['Country or Area', 'Year'])


# 选择相关的列：国家代号（NOC）、年份（Year）和GDP值（GDP）
gdp_for_first_medals = merged_data[['NOC', 'Year', 'GDP_Percentage']].copy()  # 使用 .copy() 来确保这是一个独立的副本

gdp_for_first_medals_1 = merged_data_1[['NOC', 'Year', 'GDP_Percentage']].copy()  # 使用 .copy() 来确保这是一个独立的副本

gdp_for_first_medals_2 = merged_data_2[['NOC', 'Year', 'GDP_Percentage']].copy()  # 使用 .copy() 来确保这是一个独立的副本

# 加载人口数据文件
population_file_path = r'data0/population_percentage.csv'
population_data = pd.read_csv(population_file_path)

# 将宽表转换为长表，方便按国家和年份匹配
population_data_long = population_data.melt(
    id_vars=['Country'],  # 保留的固定列
    var_name='Year',      # 新的年份列
    value_name='Population'  # 人口数据
)

# 确保 'Year' 列是整数类型
population_data_long['Year'] = population_data_long['Year'].astype(int)

# 遍历 gdp_for_first_medals 的每一行，添加对应的人口数据
for index, row in gdp_for_first_medals.iterrows():
    country_code = row['NOC']  # 国家代号
    year = row['Year']         # 年份
    
    # 找到匹配的国家和年份的人口数据
    population_row = population_data_long[
        (population_data_long['Country'] == country_code) &
        (population_data_long['Year'] == year)
    ]
    
    if not population_row.empty:
        population = population_row['Population'].values[0]
        # 使用 .loc 修改原数据框
        gdp_for_first_medals.loc[index, 'Population'] = population

first_medals_after_1970 = gdp_for_first_medals.rename(columns={'NOC': 'Team','Population':'Population_Percentage'})


for index, row in gdp_for_first_medals_1.iterrows():
    country_code = row['NOC']  # 国家代号
    year = row['Year']         # 年份
    
    # 找到匹配的国家和年份的人口数据
    population_row = population_data_long[
        (population_data_long['Country'] == country_code) &
        (population_data_long['Year'] == year)
    ]
    
    if not population_row.empty:
        population = population_row['Population'].values[0]
        # 使用 .loc 修改原数据框
        gdp_for_first_medals_1.loc[index, 'Population'] = population

first_medals_after_1970_1 = gdp_for_first_medals_1.rename(columns={'NOC': 'Team','Population':'Population_Percentage'})

for index, row in gdp_for_first_medals_2.iterrows():
    country_code = row['NOC']  # 国家代号
    year = row['Year']         # 年份
    
    # 找到匹配的国家和年份的人口数据
    population_row = population_data_long[
        (population_data_long['Country'] == country_code) &
        (population_data_long['Year'] == year)
    ]
    
    if not population_row.empty:
        population = population_row['Population'].values[0]
        # 使用 .loc 修改原数据框
        gdp_for_first_medals_2.loc[index, 'Population'] = population

first_medals_after_1970_2 = gdp_for_first_medals_2.rename(columns={'NOC': 'Team','Population':'Population_Percentage'})


# 6. 加载运动员数据
athletes_file_path = r'data/summerOly_athletes.csv'
athletes_data = pd.read_csv(athletes_file_path)

# 7. 查看运动员数据的列名，以确保正确的列名
print("运动员数据列名：", athletes_data.columns)

# 如果列名是 'Name'、'Sport'、'Event'，按需修改
athletes_stats = athletes_data.groupby(['Team', 'Year']).agg(
    athletes_count=('Name', 'nunique'),  # 假设运动员的名字在 'Name' 列
    sports_count=('Sport', 'nunique'),
    events_count=('Event', 'nunique')
).reset_index()

# 8. 将运动员统计数据与第一次获奖年份的数据合并
merged_data = pd.merge(first_medals_after_1970, athletes_stats, on=['Team', 'Year'], how='left')
merged_data_1 = pd.merge(first_medals_after_1970_1, athletes_stats, on=['Team', 'Year'], how='left')
merged_data_2 = pd.merge(first_medals_after_1970_2, athletes_stats, on=['Team', 'Year'], how='left')
# 为每个表格添加 Period 列
merged_data['Period'] = 'First'
merged_data_1['Period'] = 'Prev'
merged_data_2['Period'] = 'Next'

# 拼接三个表格
final_merged_data = pd.concat([merged_data, merged_data_1, merged_data_2], ignore_index=True)

# 重新排序列（可选）
final_merged_data = final_merged_data[['Period', 'Team', 'Year', 'GDP_Percentage', 'Population_Percentage',
                                       'athletes_count', 'sports_count', 'events_count']]

# 保存拼接后的表格为 CSV 文件
output_file_path = r'final_combined_data.csv'
final_merged_data.to_csv(output_file_path, index=False)

# 输出结果确认
print(f"拼接后的数据已保存到文件：{output_file_path}")


运动员数据列名： Index(['Name', 'Sex', 'Team', 'NOC', 'Year', 'City', 'Sport', 'Event',
       'Medal'],
      dtype='object')
拼接后的数据已保存到文件：final_combined_data.csv
